This code does the following:

1) Access SportsReference website to pull college team names
2) Convert these team names to usable addresses to access the logos (.png files)
3) Download all logos

In [24]:
#Pull list of all D1 schools that played in 2016-17 season
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import os

#establish variables for list/DB objects
name = []
School_list = []
Add_list = []
Col = ["SCHOOL"]
Col2 = ["ADDRESS"]
School_list.append(Col)
Add_list.append(Col2)
Src_DB = []
Rmv_Char = ['.', '(', ')', '&']
c = 0

#establish variable for HTML address
sref = 'https://www.sports-reference.com/cbb/seasons/2017-school-stats.html'

#Parse School Names from Basic Season Stats page
r = requests.get(sref)
soup = BeautifulSoup(r.content, "lxml")
t = soup.find("table", id = "basic_school_stats")

for i in t.find_all("tr")[:]:
    name = [a.get_text().encode('utf-8') for a in i.find_all("a")]
    for name in name:
        School_list.append(name)
        name = name.translate(None, ''.join(Rmv_Char)).replace("'", "").replace(" ", "-") + '.png'
        Add_list.append(name.lower())
        

#exception handling to re-write school names that do not match off with the address for the .png file        
for p in range(0, len(Add_list)-1):
    if Add_list[p] == 'siu-edwardsville.png':
        Add_list[p] = 'southern-illinois-edwardsville.png'
    elif Add_list[p] == 'texas-rio-grande-valley.png':
        Add_list[p] = 'texas-pan-american.png'
    elif Add_list[p] == 'uc-davis.png':
        Add_list[p] = 'california-davis.png'
    elif Add_list[p] == 'uc-irvine.png':
        Add_list[p] = 'california-irvine.png'
    elif Add_list[p] == 'uc-riverside.png':
        Add_list[p] = 'california-riverside.png'
    elif Add_list[p] == 'uc-santa-barbara.png':
        Add_list[p] = 'california-santa-barbara.png'
    elif Add_list[p] == 'university-of-california.png':
        Add_list[p] = 'california.png'
    elif Add_list[p] == 'vmi.png':
        Add_list[p] = 'virginia-military-institute.png'
    elif Add_list[p] == 'william--mary.png':
        Add_list[p] = 'william-mary.png'
    
Src_DB = pd.DataFrame(
    {'SCHOOL': School_list,
     'Src': Add_list
    })    

Src_DB = Src_DB[1:].reset_index(drop=True)

                    SCHOOL                              Src
0        Abilene Christian            abilene-christian.png
1                Air Force                    air-force.png
2                    Akron                        akron.png
3              Alabama A&M                   alabama-am.png
4       Alabama-Birmingham           alabama-birmingham.png
5            Alabama State                alabama-state.png
6                  Alabama                      alabama.png
7              Albany (NY)                    albany-ny.png
8             Alcorn State                 alcorn-state.png
9                 American                     american.png
10       Appalachian State            appalachian-state.png
11           Arizona State                arizona-state.png
12                 Arizona                      arizona.png
13    Arkansas-Little Rock         arkansas-little-rock.png
14     Arkansas-Pine Bluff          arkansas-pine-bluff.png
15          Arkansas State              

In [13]:
#img files are downloaded from third party site; using the below url in combination with team names from sports reference to access
img_address = str('https://d2p3bygnnzw9w3.cloudfront.net/req/201707101/tlogo/ncaa/')
for s in Src_DB['Src'][0:]:
    u = img_address + str(s)
    h = requests.get(u)
    soup2 = BeautifulSoup(h.content, "lxml")
    image_name = str(s)
    with open(image_name, "wb") as f:
        f.write(h.content)